# MODULES

In [2]:
import panel as pn
import panel.widgets as pnw
pn.extension('plotly') #para que nos despliegue en nuestro notebook, activamos la extension de panel

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math

# CLASSES

In [8]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

## Euclidian Distance

In [3]:
def dist_eucl(p1, p2):
  dist_E = math.sqrt(sum([(x - y)** 2 for x, y in zip(p1, p2)]))
  return dist_E

# TRAJECTORIES

## Brownian Motion (BM)

In [3]:
def BM(n_steps = 1000, speed = 6, s_x_pos = 0, s_y_pos = 0):

  #Init velocity vector, La clase Vec2d nos sirve como herramienta para definir el vector de velocidad que nos dice en que direccion y a que rapidez nos estamos moviendo
  velocity = Vec2d(speed,0) #<- cuando ya tenemos la rapidez en la que nos queremos mover, podemos definir este vector de velocidad con 
  #una instancia de la clase vec2d q inicia como un vector horizontal teniendo x=6 & y=0 

  #Init DataFrame
  BM_df = pd.DataFrame(columns= ['x_pos', 'y_pos'])

  #DF aux para cargar la pos inicial
  temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])# le mandamos un diccionario con los valores de la posicion x & y
  
  #concatenamos el df actual y el DF aux(temp), ignore_index=True para que no tome en cuenta los indices y no genere uno nuevo que inicie en 0 
  BM_df = pd.concat([BM_df, temp_df], ignore_index=True)#mete la primer posicion del agente
  
  #para poblarlo generamos los valores aleatorios de los giros
  for i in range(n_steps-1):
    #generamos los giros para BM a partir de una distribucion uniforme
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi) 
    velocity = velocity.rotated(turn_angle) #giramos el vector de velocidad segun el valor aleatorio asignado por turn angle 

    temp_df = pd.DataFrame([{'x_pos': BM_df.x_pos[i]+velocity.x, 'y_pos':BM_df.y_pos[i]+velocity.y}])
    BM_df= pd.concat([BM_df, temp_df], ignore_index=True)

  #return DF
  return BM_df


# PANEL WIDGETS

In [1]:
radioButtons = pn.widgets.RadioButtonGroup(options=['BM', 'CRW', 'LF'])

#Sliders
nSteps = pnw.IntSlider(name='Number of steps', width=300, value=50, step=10, start=0, end=1000)
speed = pnw.IntSlider(name='Speed', width=300, value=3, step=1, start=0, end=6)

#IntImput
xInitPos = pnw.IntInput(name='xInitPos', width=100, value=0, step=1,start=1, end=50)
yInitPos = pnw.IntInput(name='yInitPos', width=100, value=0, step=1,start=1, end=50)

#Select
metricsType = pnw.Select(name='Metrics type', value='MSD', options=['MSD', 'PL'])

@pn.depends(radioButtons,speed,nSteps,xInitPos,yInitPos)
def params(x):
  if x == 'BM':
    bmRowCol = pn.Column(
        pn.Row('',nSteps),
        pn.Row(
            pn.Column('',xInitPos),
            pn.Spacer(width=50),
            pn.Column('',yInitPos)
        ),
        pn.Row('',speed),
        pn.Row('',metricsType)
    )
    return bmRowCol
  elif x == 'CRW':
    bmRowCol = pn.Column(
        pn.Row('',nSteps),
        pn.Row(
        pn.Column('',xInitPos),
        pn.Spacer(width=50),
        pn.Column('',yInitPos)
    ),
      pn.Row('',speed)  
    )
    return bmRowCol 
  elif x == 'LF':
    bmRowCol = pn.Column(
        pn.Row('',nSteps),
        pn.Row(
        pn.Column('',xInitPos),
        pn.Spacer(width=50),
        pn.Column('',yInitPos)
    ),
      pn.Row('',speed)  
    )
    return bmRowCol 

# create interaction between widget and function
pn.interact(params, x=radioButtons)

NameError: name 'pn' is not defined